In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('drive/MyDrive')
os.listdir('src')

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive'

In [5]:
import torch
from torchsummary import summary
from src.utils import load_data
from src.transformations import pretrained_transform
from src.models.pretrained_models import ResNetPretrained
from src.model_trainer import ModelTrainer
import itertools

In [6]:
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
device
torch.manual_seed(123)

In [7]:
torch.set_num_threads(2)

In [8]:
# cell for donwloading data

import os
import tarfile
import urllib.request
import shutil

download_dir = "/data"

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

tar_path = os.path.join(download_dir, "CINIC-10.tar.gz")
cinic10_url = "https://datashare.is.ed.ac.uk/bitstream/handle/10283/3192/CINIC-10.tar.gz"
urllib.request.urlretrieve(cinic10_url, tar_path)
print("Finished downloading")

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=download_dir)

extracted_dir = os.path.join(download_dir, "CINIC-10")

if os.path.exists(extracted_dir) and os.path.isdir(extracted_dir):
    for item in os.listdir(extracted_dir):
        src_path = os.path.join(extracted_dir, item)
        dst_path = os.path.join(download_dir, item)
        shutil.move(src_path, dst_path)

print(download_dir)

Finished downloading
/data


In [ ]:
train_loader = pretrained_transform(
    "/data/train",
    batch_size=128,
    shuffle=True,
    transform=pretrained_transform(),
    num_workers=2,
)

In [10]:
n_epochs = 10
max_batches = None

In [11]:
test_loader = load_data('/data/test', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)
train_loader = load_data('/data/train', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)
valid_loader = load_data('/data/valid', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)

In [ ]:
training_process_grid = {
    "optimizer_type": ["adam", "sgd"],
    "learning_rate": [0.01, 0.001, 0.0001],
}

regularization_grid = {
    "weight_decay": [0.01, 0.001, 0.0001],
    "dropout": [0.2, 0.5, 0.7],
}

# grid search for training parameters with set L1 regularizer = 0 & dropout = 0
# grid search for regularization parameters with set optimizer = adam & learning_rate = 0.001

training_param_combinations = list(itertools.product(*training_process_grid.values()))
regularization_param_combinations = list(
    itertools.product(*regularization_grid.values())
)

In [ ]:
learning_rate = 0.001
for optimizer_type in ["adam", "sgd"]:
    resnet = ResNetPretrained()
    learning_rate_str = f"{learning_rate}".replace("0.", "")
    trainer = ModelTrainer(
        model=resnet,
        train_loader=train_loader,
        device=device,
        weight_decay=0,
        optimizer_type=optimizer_type,
        learning_rate=learning_rate,
        log_file=f"resnet_{optimizer_type}_{learning_rate}.json",
        save_dir=f"./saved_models/resnet/training_params_{optimizer_type}_{learning_rate_str}",
        max_batches=max_batches,
        valid_loader=valid_loader
    )
    trainer.train(n_epochs)

is cuda False
Batch count 0


In [ ]:
optimizer_type = "sgd"
for learning_rate in [0.01, 0.001, 0.0001]:
    resnet = ResNetPretrained()
    learning_rate_str = f"{learning_rate}".replace("0.", "")
    trainer = ModelTrainer(
        model=resnet,
        train_loader=train_loader,
        device=device,
        weight_decay=0,
        optimizer_type=optimizer_type,
        learning_rate=learning_rate,
        log_file=f"resnet_{optimizer_type}_{learning_rate}.json",
        save_dir=f"./saved_models/resnet/training_params_{optimizer_type}_{learning_rate_str}",
        max_batches=max_batches,
        valid_loader=valid_loader
    )
    trainer.train(n_epochs)

In [ ]:
for weight_decay, dropout in regularization_param_combinations:
    trainer = ModelTrainer(
        model=resnet,
        train_loader=train_loader,
        device=device,
        optimizer_type="adam",
        learning_rate=0.001,
        weight_decay=weight_decay,
        dropout=dropout,
        log_file=f"resnet_{weight_decay}_{dropout}.json",
        save_dir="./saved_models/resnet/regularization_params",
        max_batches=max_batches,
    )
    trainer.train(n_epochs)

is cuda False
Batch count 0
Epoch 1/1, Loss: 5.0680, Accuracy: 0.3203, F1-score: 0.2255
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_params\resnet_0.01_0.2.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 3.8199, Accuracy: 0.3398, F1-score: 0.2342
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_params\resnet_0.01_0.5.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 2.7944, Accuracy: 0.3984, F1-score: 0.2638
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_params\resnet_0.01_0.7.json
is cuda False
Batch count 0
Epoch 1/1, Loss: 2.2295, Accuracy: 0.4062, F1-score: 0.2750
Model saved to ./saved_models/resnet/regularization_params\ResNetPretrained_1.pth
Training log saved to ./saved_models/resnet/regularization_

### Best model to be tested on dataset altered through data augmentation techniques

In [ ]:
train_rotated = load_data(
    "./data/train",
    batch_size=256,
    shuffle=True,
    transform=random_rotation_transform(),
    num_workers=2,
)